# Pi Data Query

### Libraries

In [ ]:
import os
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
import time
import PIconnect as PI
from PIconnect.PIConsts import SummaryType
import math
from itertools import product
from tqdm import tqdm

### Dataframe Acquisition

In [ ]:
#Captures the file adress
path = os.getcwd()
datainput=pd.read_excel(path + '/TAGS_ROLLING.xlsx')
datainput=datainput[pd.isnull(datainput.Tags) == False]

### User input for plant and interval selection

In [ ]:
#Lists unique values from the sheet
plant_list=list(np.unique(datainput.Plant))

#Appends All Plants option
plant_list.append('All plants')

#Prints the Plant 
print('Type one of the options below to select a plant.')
for plant in plant_list:
    print(plant_list.index(plant),'. ', plant_list[plant_list.index(plant)])

#Awaits for the user to select the plant to download data
plant_code=int(input('Select an option: (Ex.: 0, 1, 2, ... ) \n'))

#Checks if the user selected All Plants or a specific one
if plant_code == len(plant_list)-1:
    print('Uploading tag list')

else:
    print('Uploading tag list')
    datainput=datainput[datainput.Plant==plant_list[plant_code]]
    datainput.reset_index(inplace=True)

#Function to replace blank bad data to nan
def elim_text(val):
    if (type(val) is float):
        a=val
    else:
        a=np.nan
    return a

#Awaits user to insert a date value
start_date=str(input("Insert starting date: (Ex.: dd/mm/yyyy) \n"))

#Awaits user to insert a date value
end_date=str(input("Insert end date: (Ex.: dd/mm/yyyy) \n"))

#Sets the variable to a date type object
start_date=datetime.strptime(start_date,"%d/%m/%Y")

#Sets the variable to a date type object
end_date=datetime.strptime(end_date,"%d/%m/%Y")

#Sets start date as a string (required by PI to perform the query)
start_date=datetime.strftime(start_date, "%d/%m/%Y")
end_date=datetime.strftime(end_date, "%d/%m/%Y")

#Sets time between each queried value
interval='10m'
variables=list(datainput.Tags)

### PI Data Download

In [ ]:
print('Connecting to PI', variables[0].split("\\")[2])

#Creates the PI Data Store Dataframe
points=[]

#Sets timezone
PI.PIConfig.DEFAULT_TIMEZONE = 'America/Sao_Paulo'

#Assign the PI TAGs to the list
with PI.PIServer(variables[0].split("\\")[2]) as server:
    for ii in range(0,len(variables)):
        points = points+server.search(variables[ii].split("\\")[3])

#Data download
print(str(len(points)), 'Tags:', "from ", start_date,"to", end_date)
start=time.time()
data1=0
data2=0
# tqdm or Taqadum shows the progress bar associated with the loop
for ii in tqdm(range(0,round(len(points))), colour='red', desc="Download Progress"):
    error_cycle=0
    while error_cycle<=10:
        try:

            #Queries the data in PI Server based in the time window, interval and summary type
            data1 = points[ii].summaries(start_date, end_date, interval, SummaryType.AVERAGE) 
            data1.reset_index(inplace=True)

            #Selects the WTG and variable name from the Excel file
            variable_name=str(datainput.WTG_SAS[ii]) + " - " + str(datainput.VAR[ii])
            data1.columns=['Time',variable_name]

            #Apply to every data in column 'variable_name' the data treatment function
            data1[variable_name]=data1.apply(lambda x: elim_text(x[variable_name]),axis=1)

            #Stores the data frame to date2 with the timestamp as index
            if ii==0:
                data2=data1
            else:
                data2=pd.merge(data2,data1, on='Time')
            break
        except:
            error_cycle+=1
            print('error')
            pass
        else:
            break

### Process conclusion and output

In [ ]:
#Saves file and finishes the process
print('Saving file')
start_date = start_date.replace("/", ".")
end_date = end_date.replace("/", ".")
data2.to_csv(path + '/PI Data Query - ' + plant_list[plant_code] + ' ' + start_date + '-' + end_date +'.csv')
print('Calculation done, file saved in ' + path)
input('Press enter to exit')